## Instructions 

The main objective of this pipeline is to create a single analytical table where each row is a flight, enriched with:
Core scheduling and routing information (carrier, origin/destination, distance, calendar fields).
Binary delay labels and supporting delay metrics (`DEP_DEL15`, `ARR_DEL15`, `DEP_DELAY`, `ARR_DELAY`).
“As-of” origin weather features sampled from NOAA hourly data before departure (temperature, visibility, wind, precipitation, etc.).
Geographic helper fields (lat/lon for airports and stations, station–airport distances).
The result is a model-ready dataset that can be used for classification (on-time vs delayed) regression on delay minutes, while carefully avoiding information leakage from post-departure variables.

### Feature selection and leakage control

We define a clear feature-selection and leakage-control policy before any heavy joins are performed. First, it distinguishes outcomes from inputs: delay outcomes are treated strictly as labels or evaluation targets, while everything that could realistically be known before departure is treated as candidate predictors. On the input side, the dataset emphasizes variables that are fixed in advance or known operationally before takeoff: calendar information that captures seasonal and weekly patterns; planned schedule details such as planned departure and arrival times and planned flight duration; and structural flight characteristics like carrier identity, route, and distance. These are complemented with “as-of” origin weather features obtained from historical observations up to a fixed cutoff time before departure, ensuring that only past or present meteorological information is used. In contrast, any field that reflects what actually happened after the aircraft began its departure sequence—such as realized delays, taxi-out times, airborne times, or ex-post cause codes—is explicitly excluded from the modeling feature set to avoid data leakage. Those variables are retained only in separate diagnostic groups for analysis and model evaluation, not as predictors. This separation between pre-event and post-event information ensures that the final feature set mirrors the information a real-time system would have at prediction time, guarding against over-optimistic performance estimates and improving the robustness of the resulting models.

| **Feature**           | **Type**           | **Description**                                         | **Notes / Use in Model**                     |
| --------------------- | ------------------ | ------------------------------------------------------- | -------------------------------------------- |
| `DEP_DEL15`           | Target (Binary)    | 1 if departure delay ≥15 minutes, else 0                | Main prediction target                       |
| `DEP_DELAY`           | Numeric            | Departure delay in minutes (negative = early departure) | Used for validation and correlation          |
| `CRS_DEP_HOUR`        | Numeric (0–23)     | Scheduled departure hour extracted from `CRS_DEP_TIME`  | Captures daily delay pattern                 |
| `DAY_OF_WEEK`         | Categorical (1–7)  | Day of the week (1 = Monday, 7 = Sunday)                | Reflects weekday vs. weekend trends          |
| `MONTH`               | Categorical (1–12) | Month of the year                                       | Captures seasonal variation                  |
| `OP_UNIQUE_CARRIER`   | Categorical        | Airline carrier code (e.g., AA, DL, UA)                 | Delay rates vary by airline                  |
| `ORIGIN`              | Categorical        | Origin airport IATA code                                | Captures airport-level delay patterns        |
| `DEST`                | Categorical        | Destination airport IATA code                           | May add contextual variation                 |
| `DISTANCE`            | Numeric            | Flight distance in miles                                | Longer flights less affected by short delays |
| `TAXI_OUT`            | Numeric            | Taxi-out time in minutes                                | Indicator of airport congestion              |
| `CRS_ELAPSED_TIME`    | Numeric            | Scheduled flight duration in minutes                    | Useful for normalization                     |
| `HourlyPrecipitation` | Numeric            | Precipitation at departure station (inches/hour)        | Proxy for adverse weather                    |
| `HourlyVisibility`    | Numeric            | Visibility at departure station (miles)                 | Lower values may increase delay risk         |
| `HourlyWindSpeed`     | Numeric            | Wind speed at departure station (mph)                   | Captures storm or runway condition impact    |
| `CANCELLED`           | Binary             | 1 if flight was canceled                                | May need to exclude or treat separately      |
| `DIVERTED`            | Binary             | 1 if flight diverted to another airport                 | Usually excluded for modeling                |

| Column                | Description                                                  | Source | Notes                                       |
| :-------------------- | :----------------------------------------------------------- | :----- | :------------------------------------------ |
| QUARTER               | Calendar quarter of the year (1–4)                           | Flight | some seasons might experience more delays than others: seasonality                      |
| MONTH                 | Month of flight date (1–12)                                  | Flight | Use for monthly trends                      |
| DAY_OF_MONTH          | Day of the month (1–31)                                      | Flight | Temporal feature                            |
| DAY_OF_WEEK           | Day of the week (1=Mon, 7=Sun)                               | Flight | Delays vary by weekday                      |
| FL_DATE               | Flight date                                                  | Flight | Combine with time fields for timestamp      |
| OP_UNIQUE_CARRIER     | Unique airline carrier code (e.g., AA, DL)                   | Flight | Key categorical variable                    |
| OP_CARRIER_AIRLINE_ID | Airline numeric ID from BTS                                  | Flight | Alternate carrier ID                        |
| OP_CARRIER            | Carrier abbreviation                                         | Flight | Duplicate of OP_UNIQUE_CARRIER              |
| TAIL_NUM              | Aircraft tail number                                         | Flight | Often missing or reused                     |
| OP_CARRIER_FL_NUM     | Flight number                                                | Flight | Combine with carrier for unique flight ID   |
| ORIGIN_AIRPORT_ID     | Unique numeric ID for origin airport                         | Flight | Key for joins                               |
| ORIGIN_AIRPORT_SEQ_ID | Unique ID per airport sequence                               | Flight | Not needed for modeling                     |
| ORIGIN_CITY_MARKET_ID | City market ID                                               | Flight | Identifies metro area                       |
| ORIGIN                | Origin airport code (IATA)                                   | Flight | Major key feature                           |
| ORIGIN_CITY_NAME      | Full city name of origin                                     | Flight | Redundant with ORIGIN                       |
| ORIGIN_STATE_ABR      | Origin state abbreviation                                    | Flight | Useful for mapping                          |
| ORIGIN_STATE_FIPS     | State FIPS code                                              | Flight | Redundant geographic ID                     |
| ORIGIN_STATE_NM       | Full state name                                              | Flight | Informational only                          |
| ORIGIN_WAC            | World Area Code for origin                                   | Flight | May be dropped                              |
| DEST_AIRPORT_ID       | Unique numeric ID for destination airport                    | Flight | Key for joins                               |
| DEST_AIRPORT_SEQ_ID   | Destination sequence ID                                      | Flight | Often redundant                             |
| DEST_CITY_MARKET_ID   | Destination city market ID                                   | Flight | Identifies metro area                       |
| DEST                  | Destination airport code (IATA)                              | Flight | Key feature                                 |
| DEST_CITY_NAME        | Full city name of destination                                | Flight | Informational                               |
| DEST_STATE_ABR        | Destination state abbreviation                               | Flight | Useful for mapping                          |
| DEST_STATE_FIPS       | Destination state FIPS code                                  | Flight | Redundant                                   |
| DEST_STATE_NM         | Full destination state name                                  | Flight | Informational only                          |
| DEST_WAC              | World Area Code for destination                              | Flight | May be dropped                              |
| CRS_DEP_TIME          | Scheduled departure time (HHMM local)                        | Flight | Convert to hour for modeling                |
| DEP_TIME              | Actual departure time (HHMM local)                           | Flight | Post-departure → leakage                    |
| DEP_DELAY             | Departure delay in minutes                                   | Flight | Leakage (after event)                       |
| DEP_DELAY_NEW         | Departure delay, no negatives                                | Flight | Leakage (after event)                       |
| DEP_DEL15             | 1 if departure delay ≥15 min                                 | Flight | Post-departure indicator                    |
| DEP_DELAY_GROUP       | Categorical group of departure delay                         | Flight | Leakage variable                            |
| DEP_TIME_BLK          | Scheduled departure block (time interval)                    | Flight | Keep for modeling                           |
| TAXI_OUT              | Taxi-out time in minutes                                     | Flight | Leakage; occurs after departure             |
| WHEELS_OFF            | Time wheels left ground (HHMM)                               | Flight | Leakage                                     |
| WHEELS_ON             | Time wheels touched down (HHMM)                              | Flight | Leakage                                     |
| TAXI_IN               | Taxi-in time (minutes)                                       | Flight | Leakage                                     |
| CRS_ARR_TIME          | Scheduled arrival time (HHMM)                                | Flight | Keep; pre-scheduled info                    |
| ARR_TIME              | Actual arrival time (HHMM)                                   | Flight | Leakage                                     |
| ARR_DELAY             | Arrival delay (minutes)                                      | Flight | Target-related; drop                        |
| ARR_DELAY_NEW         | Non-negative arrival delay                                   | Flight | Redundant                                   |
| ARR_DEL15             | (1 if arrival delay ≥15 min)             | Flight | Binary label                                |
| ARR_DELAY_GROUP       | Grouped arrival delay                                        | Flight | Redundant with ARR_DEL15                    |
| ARR_TIME_BLK          | Scheduled arrival block                                      | Flight | Pre-scheduled; usable                       |
| CANCELLED             | 1 if flight was cancelled                                    | Flight | Keep for classification                     |
| CANCELLATION_CODE     | Code for reason of cancellation (A=Carrier, B=Weather, etc.) | Flight | Important categorical for cancelled flights |
| DIVERTED              | 1 if flight diverted to another airport                      | Flight | Keep; rare event                            |
| CRS_ELAPSED_TIME      | Scheduled elapsed flight time (min)                          | Flight | Useful duration variable                    |
| ACTUAL_ELAPSED_TIME   | Actual total flight time (min)                               | Flight | Leakage                                     |
| AIR_TIME              | In-air flight time (min)                                     | Flight | Leakage                                     |
| FLIGHTS               | Number of flights (usually 1)                                | Flight | Constant; drop                              |
| DISTANCE              | Great circle distance (miles)                                | Flight | Key continuous variable                     |
| DISTANCE_GROUP        | Distance category (1=short haul, etc.)                       | Flight | Categorical; keep                           |
| CARRIER_DELAY         | Delay due to airline (min)                                   | Flight | Post-event; leakage                         |
| WEATHER_DELAY         | Delay due to weather (min)                                   | Flight | Leakage                                     |
| NAS_DELAY             | Delay due to air traffic control (min)                       | Flight | Leakage                                     |
| SECURITY_DELAY        | Delay due to security (min)                                  | Flight | Leakage                                     |
| LATE_AIRCRAFT_DELAY   | Delay due to late incoming aircraft (min)                    | Flight | Leakage                                     |
| FIRST_DEP_TIME        | First departure attempt (for multi-leg flights)              | Flight | Leakage                                     |
| TOTAL_ADD_GTIME       | Total gate time added                                        | Flight | Leakage                                     |
| LONGEST_ADD_GTIME     | Longest gate time added                                      | Flight | Leakage                                     |

| Category                           | Columns                                                                                                                                                                                               | Action                       | Notes                                                                             |
| :--------------------------------- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :--------------------------- | :-------------------------------------------------------------------------------- |
| **Target**                         | `ARR_DEL15`                                                                                                                                                                                           | Keep                         | Binary classification label (1 = delay ≥15 min)                                   |
| **Temporal Features**              | `MONTH`, `DAY_OF_WEEK`, `CRS_DEP_TIME`, `CRS_ARR_TIME`, `DEP_TIME_BLK`, `ARR_TIME_BLK`                                                                                                                | Keep (or engineer)           | Convert times to hour bins or features; captures time-of-day and seasonal effects |
| **Flight Ops / Routing**           | `OP_UNIQUE_CARRIER`, `ORIGIN`, `DEST`, `DISTANCE`, `DISTANCE_GROUP`, `CANCELLED`, `DIVERTED`                                                                                                          | Keep                         | Core predictive variables; encode categorical features                            |
| **Weather (Hourly)**               | `HourlyPrecipitation`, `HourlyVisibility`, `HourlyWindSpeed`                                                                                                                                          | Keep                         | Key real-time weather indicators                                                  |
| **Weather (Hourly – Derived)**     | `HourlyPrecipitation_D`, `HourlyVisibility_D`, `HourlyWindSpeed_D`                                                                                                                                    | Keep                         | Change/delta features showing recent shifts                                       |
| **Weather (Daily)**                | `DailyPrecipitation`, `DailyMaximumDryBulbTemperature`, `DailyMinimumDryBulbTemperature`, `DailyPeakWindSpeed`                                                                                        | Review                       | Optional aggregates for extended weather context                                  |
| **Geographic / Station Info**      | `LATITUDE`, `LONGITUDE`, `ELEVATION`, `origin_station_dis`                                                                                                                                            | Review                       | Keep if you plan spatial or distance-based analysis                               |
| **Carrier ID / Meta**              | `OP_CARRIER_AIRLINE_ID`, `OP_CARRIER`                                                                                                                                                                 | Drop                         | Redundant with `OP_UNIQUE_CARRIER`                                                |
| **Delay & Post-Arrival Variables** | `DEP_DELAY`, `ARR_DELAY`, `AIR_TIME`, `ACTUAL_ELAPSED_TIME`, `WHEELS_OFF`, `WHEELS_ON`, `TAXI_OUT`, `TAXI_IN`, `CARRIER_DELAY`, `WEATHER_DELAY`, `NAS_DELAY`, `SECURITY_DELAY`, `LATE_AIRCRAFT_DELAY` | Drop (leakage)               | All occur after or depend on delay outcome                                        |
| **Monthly / Climate Features**     | All `Monthly...` fields (`MonthlyAverageRH`, `MonthlyGreatestPrecip`, etc.)                                                                                                                           | Drop                         | More than 90% null and not relevant for single-flight prediction                  |
| **High-Null / Empty Monthly & Short-Duration** | `MonthlyAverageRH`, `MonthlyDaysWithGT001Precip`, `MonthlyDaysWithGT010Precip`, `MonthlyDaysWithGT32Temp`, `MonthlyDaysWithGT90Temp`, `MonthlyDaysWithLT0Temp`, `MonthlyDaysWithLT32Temp`, `MonthlyDepartureFromNormalAverageTemperature`, `MonthlyDepartureFromNormalCoolingDegreeDays`, `MonthlyDepartureFromNormalHeatingDegreeDays`, `MonthlyDepartureFromNormalMaximumTemperature`, `MonthlyDepartureFromNormalMinimumTemperature`, `MonthlyDepartureFromNormalPrecipitation`, `MonthlyDewpointTemperature`, `MonthlyGreatestPrecip`, `MonthlyGreatestPrecipDate`, `MonthlyGreatestSnowDepth`, `MonthlyGreatestSnowDepthDate`, `MonthlyGreatestSnowfall`, `MonthlyGreatestSnowfallDate`, `MonthlyMaxSeaLevelPressureValue`, `MonthlyMaxSeaLevelPressureValueDate`, `MonthlyMaxSeaLevelPressureValueTime`, `MonthlyMaximumTemperature`, `MonthlyMeanTemperature`, `MonthlyMinSeaLevelPressureValue`, `MonthlyMinSeaLevelPressureValueDate`, `MonthlyMinSeaLevelPressureValueTime`, `MonthlyMinimumTemperature`, `MonthlySeaLevelPressure`, `MonthlyStationPressure`, `MonthlyTotalLiquidPrecipitation`, `MonthlyTotalSnowfall`, `MonthlyWetBulb`, `AWND`, `CDSD`, `CLDD`, `DSNW`, `HDSD`, `HTDD`, `NormalsCoolingDegreeDay`, `NormalsHeatingDegreeDay`, `ShortDurationEndDate005`, `ShortDurationEndDate010`, `ShortDurationEndDate015`, `ShortDurationEndDate020`, `ShortDurationEndDate030`, `ShortDurationEndDate045`, `ShortDurationEndDate060`, `ShortDurationEndDate080`, `ShortDurationEndDate100`, `ShortDurationEndDate120`, `ShortDurationEndDate150`, `ShortDurationEndDate180`, `ShortDurationPrecipitationValue005`, `ShortDurationPrecipitationValue010`, `ShortDurationPrecipitationValue015`, `ShortDurationPrecipitationValue020`, `ShortDurationPrecipitationValue030`, `ShortDurationPrecipitationValue045`, `ShortDurationPrecipitationValue060`, `ShortDurationPrecipitationValue080`, `ShortDurationPrecipitationValue100`, `ShortDurationPrecipitationValue120`, `ShortDurationPrecipitationValue150`, `ShortDurationPrecipitationValue180` | Drop   | All null in OTPW;  safe to drop to simplify schema |
| **Backup / Metadata Fields**       | All `Backup...`, `ShortDuration...`, `_row_desc`, `REM`                                                                                                                                               | Drop                         | Join metadata and reference only                                                  |
| **Text Fields**                    | `NAME`, `REPORT_TYPE`, `DailyWeather`, `HourlySkyConditions`, `HourlyPresentWeatherType`                                                                                                              | Optional (drop for baseline) | Free text; may require NLP or feature extraction later                            |
| **Station Linking**                | `STATION`, `DATE`, `SOURCE`                                                                                                                                                                           | Keep for validation only     | Needed for join checks; not a model input                                         |

#### iv. Airport–Weather Integration Plan (Data Joins, Issues & Rationale)

Our exploratory review of the four source tables (flights, airport codes, weather, and station metadata) showed that the **biggest blockers are not in the flights themselves but in the lookup tables we need to join to**. The flights table already carries clean IATA airport codes (`ORIGIN`, `DEST`) and consistent date fields (`FL_DATE`, `YEAR`, `MONTH`, …), so it is a good factual backbone. However, our main airport codes file does **not** include time zones and sometimes only stores geolocation as a single `coordinates` string (`"lon, lat"`). At the same time, the external GitHub airport list **does** provide `timezone`, and often better lat/lon, but it doesn’t perfectly align 1:1 with our current codes file. This makes a direct “flights → airports → weather” join fragile, because we would be mixing two partially-overlapping airport catalogs. To fix this, we will first build **one master airport dimension** by joining the GitHub timezones and geolocation into the codes we already use in flights, and we will coalesce coordinates so that every IATA that appears as an origin/destination ends up with: **(a)** a timezone, **(b)** a lat/lon pair, and **(c)** a human-readable name.

The second major issue is at the **weather** side: NOAA data is hourly and station-based (`STATION`, `DATE`, 100+ weather features), not airport-based. That means there is no native key to connect “ATL, SFO, ORD…” directly to a weather row. Also, stations and airports don’t share the exact same IDs: stations use a different identifier (and sometimes we need to normalize with the `stations.csv` file). On top of that, **weather is in UTC** while our flights are in **local airport time**, so if we don’t add airport timezones first, we can’t reliably pick “the weather hour that corresponds to this departure.” To solve this, we will: (1) compute airport → nearest-(1..3)-station pairs using the unified lat/lon we just created, (2) store that in a small bridge table (`airport_weather_station`), and (3) when we enrich flights, we will convert flight times to UTC using the airport’s timezone and then pick the matching hourly weather from the correct station. This approach gives us a repeatable pattern we can scale from the 3-month sample up to 2015–2021.

**Key data problems we identified:**
- Our original airport codes file **lacks time zones**, so flight local times cannot be aligned to UTC weather.
- Airport geolocation is sometimes packed as a single text field (`coordinates`), so we must **parse and standardize lat/lon**.
- Weather rows are **station-based, not airport-based**, so we must create an extra **airport → station** bridge.
- Stations may come from **two slightly different sources** (`weather.csv` vs `stations.csv`), so we need **ID normalization**.
- Not all flights’ origin/dest codes are guaranteed to appear in the GitHub airport list, so we will **fallback to the original codes file** to avoid losing rows.

#### v. Entity–Relationship Blueprint for Flights ↔ Airports ↔ Weather

This diagram summarizes the core entities we will use to enrich flight records with meteorological data.


In [0]:
mermaid_diagram_joins = """
<div class="mermaid">
erDiagram
    %% LEGEND
    %% PK_... = primary key (or business key)
    %% FK_... = foreign key to another table

    FLIGHTS {
        string PK_flight_row
        date FL_DATE
        string FK_origin_iata_code
        string FK_dest_iata_code
        string OP_UNIQUE_CARRIER
        int OP_CARRIER_FL_NUM
        int CRS_DEP_TIME
        int CRS_ARR_TIME
        int YEAR
        int MONTH
        int DAY_OF_MONTH
    }

    MASTER_AIRPORTS {
        string PK_iata_code
        string ident
        string name
        string municipality
        string iso_country
        string iso_region
        string airport_timezone
        float lat
        float lon
    }

    WEATHER {
        string PK_station_id
        datetime PK_obs_datetime
        float LATITUDE
        float LONGITUDE
        string NAME
        float HourlyDryBulbTemperature
        float HourlyVisibility
        float HourlyWindSpeed
    }

    NOAA_STATIONS {
        string PK_station_id_norm
        float lat
        float lon
        string neighbor_id
        float distance_to_neighbor
    }

    AIRPORT_WEATHER_STATION {
        string PK_iata_code
        string PK_station_id
        int PK_rank
        float dist_km
    }

    CHECKPOINTS {
        string checkpoint_name
        string file_path
        int rows
        int columns
        string description
    }

    %% RELATIONSHIPS
    FLIGHTS }o--|| MASTER_AIRPORTS : "origin (FK_origin_iata_code)"
    FLIGHTS }o--|| MASTER_AIRPORTS : "destination (FK_dest_iata_code)"
    MASTER_AIRPORTS ||--o{ AIRPORT_WEATHER_STATION : "airport → nearest stations"
    WEATHER ||--o{ AIRPORT_WEATHER_STATION : "station in bridge"
    WEATHER }o--|| NOAA_STATIONS : "normalize/enrich station"
    FLIGHTS ||--o{ CHECKPOINTS : "pipeline stages"
</div>
<script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
<script>mermaid.initialize({startOnLoad:true});</script>
"""
displayHTML(mermaid_diagram_joins)

erDiagram
 %% LEGEND
 %% PK_... = primary key (or business key)
 %% FK_... = foreign key to another table

 FLIGHTS {
 string PK_flight_row
 date FL_DATE
 string FK_origin_iata_code
 string FK_dest_iata_code
 string OP_UNIQUE_CARRIER
 int OP_CARRIER_FL_NUM
 int CRS_DEP_TIME
 int CRS_ARR_TIME
 int YEAR
 int MONTH
 int DAY_OF_MONTH
 }

 MASTER_AIRPORTS {
 string PK_iata_code
 string ident
 string name
 string municipality
 string iso_country
 string iso_region
 string airport_timezone
 float lat
 float lon
 }

 WEATHER {
 string PK_station_id
 datetime PK_obs_datetime
 float LATITUDE
 float LONGITUDE
 string NAME
 float HourlyDryBulbTemperature
 float HourlyVisibility
 float HourlyWindSpeed
 }

 NOAA_STATIONS {
 string PK_station_id_norm
 float lat
 float lon
 string neighbor_id
 float distance_to_neighbor
 }

 AIRPORT_WEATHER_STATION {
 string PK_iata_code
 string PK_station_id
 int PK_rank
 float dist_km
 }

 CHECKPOINTS {
 string checkpoint_name
 string file_path
 int rows
 int columns
 string description
 }

 %% RELATIONSHIPS
 FLIGHTS }o--|| MASTER_AIRPORTS : "origin (FK_origin_iata_code)"
 FLIGHTS }o--|| MASTER_AIRPORTS : "destination (FK_dest_iata_code)"
 MASTER_AIRPORTS ||--o{ AIRPORT_WEATHER_STATION : "airport → nearest stations"
 WEATHER ||--o{ AIRPORT_WEATHER_STATION : "station in bridge"
 WEATHER }o--|| NOAA_STATIONS : "normalize/enrich station"
 FLIGHTS ||--o{ CHECKPOINTS : "pipeline stages"

#### vi. To-Do List for Airport–Weather Join Pipeline

**Phase 0 – Staging (raw → stg)**  
- [ ] Load **flights** (3m / 6m / 1y sample) into `stg_flights`  
- [ ] Load **airport codes** (current `codes.csv`) into `stg_airport_codes`  
- [ ] Load **GitHub airports with timezone** into `stg_airport_tz`  
- [ ] Load **weather** (NOAA hourly) into `stg_weather_hourly`  
- [ ] Load **station metadata** (`stations.csv`) into `stg_noaa_stations`

**Phase 1 – Unified airport dimension**  
- [ ] Uppercase and trim IATA codes in **both** airport sources  
- [ ] Parse `coordinates` from `stg_airport_codes` → (`codes_lat`, `codes_lon`)  
- [ ] Select from `stg_airport_tz` only the needed fields: (`iata_code`, `airport_timezone`, `gh_lat`, `gh_lon`)  
- [ ] Left-join timezone + better lat/lon into `stg_airport_codes` and **coalesce** → `dim_airport (master_airports)`  
- [ ] Compare `dim_airport` to distinct `ORIGIN` and `DEST` from flights to find **missing airports**

**Phase 2 – Weather station dimension**  
- [ ] Build `dim_weather_station` = distinct (`STATION`, `LATITUDE`, `LONGITUDE`, `NAME`) from `stg_weather_hourly`  
- [ ] Left-join to `stg_noaa_stations` to fill missing coordinates / IDs  
- [ ] Validate that all stations used in weather have lat/lon

**Phase 3 – Airport ↔ station bridge (nearest K)**  
- [ ] Broadcast `dim_airport` (only airports with lat/lon)  
- [ ] Cross-join with `dim_weather_station`  
- [ ] Compute **Haversine** distance → `dist_km`  
- [ ] Window/partition by airport and keep top **K=3** nearest stations  
- [ ] Save as `airport_weather_station (iata_code, STATION, dist_km, rank)`

**Phase 4 – Time alignment**  
- [ ] From flights, build `dep_ts_local` = `FL_DATE` + `CRS_DEP_TIME`  
- [ ] Convert `dep_ts_local` to UTC using `dim_airport.airport_timezone` (origin)  
- [ ] Repeat for arrival using destination airport  
- [ ] (Optional) Materialize `dim_date` / `dim_time` for reporting and easier joins

**Phase 5 – Final enrichment views + QA**  
- [ ] Create `v_flights_with_origin_weather`:
  - join flights → origin airport → bridge (rank=1) → weather on matching UTC hour  
- [ ] Create `v_flights_with_dest_weather`:
  - join flights → destination airport → bridge (rank=1) → weather on matching UTC hour  
- [ ] Coverage report:
  - % flights with origin weather  
  - % flights with destination weather  
  - airports with `dist_km > 300` (flag for manual review)

### Configuration

In [0]:
# CONFIG: choose data slice and IO paths

# Options: "3M", "1Y", "5Y", "FUTURE"
DATA_SLICE = "FUTURE"

DATA_CONFIG = {
    "3M": {
        # course 3-month subset
        "flights_path": "dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_3m/",
        "weather_path": "dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_3m",
        "joined_output": "dbfs:/student-groups/Group_4_4/JOINED_3M.parquet",
        # The 3M dataset is already sliced
        "max_year": None,
        "max_month": None,
    },
    "1Y": {
        # course 1-year subset
        "flights_path": "dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data_1y/",
        "weather_path": "dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data_1y/",
        "joined_output": "dbfs:/student-groups/Group_4_4/JOINED_1Y.parquet",
        "max_year": None,
        "max_month": None,
    },
    "5Y": {
        # full 2015–2021 flights + weather, but we keep only up to 2019-12
        "flights_path": "dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_airlines_data/",
        "weather_path": "dbfs:/mnt/mids-w261/datasets_final_project_2022/parquet_weather_data",
        "joined_output": "dbfs:/student-groups/Group_4_4/JOINED_5Y_2015_2019.parquet",
        "max_year": 2019,
        "max_month": 12,
    },
    "FUTURE": {
        # placeholders for future pipelines
        "flights_path": "dbfs:/student-groups/Group_4_4/future_joins/BTS_OnTime/parquet_airlines_data_2020_2024_std.parquet",
        "weather_path": "dbfs:/student-groups/Group_4_4/future_joins/NOAA/LCDv2_weather_data_2020_2024_std.parquet",
        "joined_output": "dbfs:/student-groups/Group_4_4/JOINED_FUTURE.parquet",
        "max_year": None,
        "max_month": None,
    },
}

cfg = DATA_CONFIG[DATA_SLICE]

print(f"Using configuration: {DATA_SLICE}")
for k, v in cfg.items():
    print(f"  {k}: {v}")


Using configuration: FUTURE
  flights_path: dbfs:/student-groups/Group_4_4/future_joins/BTS_OnTime/parquet_airlines_data_2020_2024_std.parquet
  weather_path: dbfs:/student-groups/Group_4_4/future_joins/NOAA/LCDv2_weather_data_2020_2024_std.parquet
  joined_output: dbfs:/student-groups/Group_4_4/JOINED_FUTURE.parquet
  max_year: None
  max_month: None


## Library imports

In [0]:
import pandas as pd
import urllib.request
import pyspark.sql.functions as sf
from pyspark.sql import Window as W
from datetime import datetime

## Load stats helpers

In [0]:
# RUN STATS: start timer & helper functions

job_start = datetime.now()
print(f"Job start: {job_start.isoformat()}")

# Ensure dbutils is available (works in Databricks)
try:
    dbutils  # type: ignore[name-defined]
except NameError:
    from pyspark.dbutils import DBUtils
    dbutils = DBUtils(spark)

def get_dir_size(path: str) -> int:
    """
    Recursively compute total size (in bytes) of all files under `path`.
    Works for DBFS paths like 'dbfs:/mnt/...'.
    """
    total = 0
    for f in dbutils.fs.ls(path):
        if f.isDir():
            total += get_dir_size(f.path)
        else:
            total += f.size
    return total

Job start: 2025-12-14T18:10:29.575959


## Load helpers

In [0]:
# Helper function to pretty print databases
def show_df(df, n=5):
    """Pretty print the first `n` rows of a Spark DataFrame using Databricks display."""
    display(df.limit(n))

# Helper function to display columns of a Spark DataFrame
def show_columns(df):
    """Display the column names, data types, and % of null values of a Spark DataFrame."""
    total_rows = df.count()
    null_counts = df.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    percent_null = {c: (null_counts[c] / total_rows * 100) if total_rows > 0 else None for c in df.columns}
    col_info = pd.DataFrame({
        "Column": df.columns,
        "Type": [t for _, t in df.dtypes],
        "% Null": [percent_null[c] for c in df.columns]
    })
    display(col_info)
    print(f"Total rows: {total_rows}")

# Helper function haversine calculation
def haversine_km_expr(lat1, lon1, lat2, lon2):
    """
    Great-circle distance on a sphere (WGS84 mean Earth radius).
    All arguments are Column[double] in radians.
    Returns Column[double] in kilometers.
    """
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = sf.pow(sf.sin(dlat / 2), 2) + sf.cos(lat1) * sf.cos(lat2) * sf.pow(sf.sin(dlon / 2), 2)
    c = 2 * sf.atan2(sf.sqrt(a), sf.sqrt(1 - a))
    return sf.lit(6371.0088) * c  # mean Earth radius in km

## Load databases

In [0]:
# Flights data

"""
Flights data

     This is a subset of the passenger flight's on-time performance data taken from the TranStats data collection available from the U.S. Department of Transportation (DOT)

        https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ 

    Links to an external site. 

The flight dataset was downloaded from the US Department of Transportation
Links to an external site. and contains flight information from 2015 to 2021
(Note flight data for the period [2015-2019] has the following dimensionality  31,746,841 x 109)
A Data Dictionary for this dataset is located here:

    https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ 
"""

# Load flights based on DATA_SLICE
df_flights = spark.read.parquet(cfg["flights_path"])

# Drop exact duplicates
n_raw = df_flights.count()
df_flights = df_flights.dropDuplicates()
n_distinct = df_flights.count()
print(f"Flights - raw rows: {n_raw:,}, distinct rows: {n_distinct:,}")

# Display Results
# show_df(df_flights, 5)
# show_columns(df_flights)


Flights - raw rows: 31,339,836, distinct rows: 31,339,836


In [0]:
# Weather data

"""
Weather table

    As a frequent flyer, we know that flight departure (and arrival)  often get affected by weather conditions, so it makes sense to collect, and process weather data corresponding to the origin and destination airports at the time of departure and arrival, respectively, and build features based upon this data. 
    The weather dataset was downloaded from the National Oceanic and Atmospheric Administration repository 

Links to an external site. and contains weather information from 2015 to 2021

    The dimensionality of the weather data for the period [2015-2019] is 630,904,436 x 177

Data dictionary (subset): 

    Please refer to pages 8-12:  https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf 

Links to an external site. 
A better version of the data dictionary can be read here: https://www.ncei.noaa.gov/pub/data/cdo/documentation/LCD_documentation.pdf
Links to an external site.

    A superset of the features is described here:

        https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf 

    Links to an external site.

A subset of the features is shared here:

    https://docs.google.com/spreadsheets/d/1v0P34NlQKrvXGCACKDeqgxpDwmj3HxaleUiTrY7VRn0/edit#gid=0 
"""

# Load weather based on DATA_SLICE
df_weather = spark.read.parquet(cfg["weather_path"])

print(f"Weather - raw rows: {df_weather.count():,}")

# Allowed hourly report types (no QCLCD daily/monthly summaries)
ALLOWED_RPT = ["FM-15", "FM-16", "FM-12"]  # METAR, SPECI, SYNOP

# Filter to allowed report types AND create obs_utc from DATE
weather_hourly = (
    df_weather
    .filter(sf.col("REPORT_TYPE").isin(ALLOWED_RPT))
    .withColumn("obs_utc", sf.col("DATE").cast("timestamp"))
)

# Preference: METAR > SPECI > SYNOP, then latest timestamp
weather_ranked = weather_hourly.withColumn(
    "report_type_rank",
    sf.when(sf.col("REPORT_TYPE")=="FM-15", 1)
     .when(sf.col("REPORT_TYPE")=="FM-16", 2)
     .when(sf.col("REPORT_TYPE")=="FM-12", 3)
     .otherwise(99)
)

win_st_hr = W.partitionBy("STATION","obs_utc") \
             .orderBy(sf.col("report_type_rank").asc(), sf.col("DATE").desc())

weather_best = (
    weather_ranked
    .withColumn("rn", sf.row_number().over(win_st_hr))
    .filter(sf.col("rn") == 1)
    .drop("rn", "report_type_rank")
)

# Display results
# show_df(df_weather, 5)
# show_columns(df_weather)


Weather - raw rows: 756,182,081


In [0]:
# Weather station data

"""
Airport dataset
    Overall the airport dataset provides some metadata about each airport.
    The airport dataset was downloaded from the US Department of Transportation and has the following dimensionality: 18,097 x 10.
    It is located here:
        dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data

"""
df_stations = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet")

# Display results
# show_df(df_stations, 5)
# show_columns(df_stations)

In [0]:
# Airport codes
"""
Airport codes Table

    Airport codes may refer to either:
         IATA airport code, a three-letter code that is used in passenger reservation, ticketing, and baggage-handling systems,
        or ICAO airport code which is a four-letter code used by ATC systems and for airports that do not have an IATA airport code (from Wikipedia).
    Here you will need to import an external airport code conversion set (source: https://datahub.io/core/airport-codes 

Links to an external site.) and join the airport codes to the airline's flights table on the IATA code (3-letter code used by passengers)
"""

# Download and load airport codes CSV to Spark DataFrame
url = "https://datahub.io/core/airport-codes/_r/-/data/airport-codes.csv"
local_path = "/tmp/airport-codes.csv"
urllib.request.urlretrieve(url, local_path)
df_codes = spark.read.format("csv").option("header", True).load(local_path)

# Display results
# show_df(df_codes, 5)
# show_columns(df_codes)

In [0]:
"""
Airport Timezone & Geolocation Table

    This external table augments airport identifiers with the fields we need
    for time alignment and spatial joins. Each record includes:
        • IATA (3-letter passenger code) and ICAO (4-letter ATC code)
        • Airport name and locality metadata
        • Latitude and longitude (decimal degrees)
        • Time zone in IANA format (e.g., "America/Los_Angeles")

    Source:
        https://raw.githubusercontent.com/lxndrblz/Airports/main/airports.csv
        (Project page: https://github.com/lxndrblz/Airports)

    Usage:
        Import this CSV and join to the flights table on IATA (3-letter code used
        in `ORIGIN` / `DEST`). Use the `timezone` column to convert local flight
        times (e.g., FL_DATE + CRS_DEP_TIME) to UTC before weather joins, and use
        `latitude`/`longitude` to compute nearest NOAA weather stations.

    Notes:
        • Normalize IATA to uppercase and drop duplicates before joining.
        • Prefer this table’s lat/lon and timezone; if an airport is missing,
          fall back to the original codes file.
        • The timezone field follows IANA; ensure your Spark build supports
          IANA names when calling `to_utc_timestamp`.
        • /tmp is ephemeral and may be cleared when the cluster shuts down; use DBFS for persistent storage.
"""

dbutils.fs.mkdirs("dbfs:/student-groups/Group_4_4")
local_path = "/dbfs/student-groups/Group_4_4/airport-zones.csv"
url = "https://raw.githubusercontent.com/lxndrblz/Airports/main/airports.csv"
urllib.request.urlretrieve(url, local_path)
df_zones = spark.read.format("csv").option("header", True).load("dbfs:/student-groups/Group_4_4/airport-zones.csv")

# Display results
# show_df(df_zones)
# show_columns(df_zones)

# Raw Dataframes Joins

## Airports

In [0]:
# Build MASTER_AIRPORTS with timezone +  latitude and longitude
codes = (
    df_codes
    .withColumn("iata_code", sf.upper("iata_code"))
    .withColumn("_coords", sf.split(sf.regexp_replace(sf.col("coordinates"), "\\s+", ""), ","))
    .withColumn("codes_lon", sf.col("_coords").getItem(0).cast("double"))
    .withColumn("codes_lat", sf.col("_coords").getItem(1).cast("double"))
    .drop("_coords")
)

# Display results
# show_df(codes, 5)



In [0]:
tz = (
    df_zones
    .withColumn("iata_code", sf.upper(sf.col("code")))
    .select(
        sf.col("iata_code"),
        sf.col("time_zone").alias("airport_timezone"),
        sf.col("latitude").cast("double").alias("gh_lat"),
        sf.col("longitude").cast("double").alias("gh_lon"),
        sf.col("name").alias("gh_name")
    )
    .dropna(subset=["iata_code"])
    .dropDuplicates(["iata_code"])
)

# Display results
# show_df(tz, 5)

In [0]:
df_airports = (
    codes.alias("c")
    .join(tz.alias("g"), on="iata_code", how="left")
    .withColumn("lat", sf.coalesce("g.gh_lat", "c.codes_lat"))
    .withColumn("lon", sf.coalesce("g.gh_lon", "c.codes_lon"))
    .withColumn("airport_timezone", sf.col("airport_timezone"))
    .select("iata_code", "ident", sf.col("c.type").alias("airport_type"), 
            "name", "municipality", "iso_country", "iso_region", "airport_timezone", "lat", "lon")
    .dropna(subset=["iata_code"])
    .dropDuplicates(["iata_code"])
)

# Display results
# show_df(df_airports, 5)

## Weather Stations

In [0]:
# Cleanup Weather stations
df_weather_station = (
    df_stations
    .select(
        sf.col("station_id").cast("string").alias("station_id"),
        sf.col("lat").cast("double").alias("lat"),
        sf.col("lon").cast("double").alias("lon")
    )
    .dropna(subset=["station_id","lat","lon"])
    .dropDuplicates(["station_id"])
)

# Display results
show_df(df_weather_station, 5)
show_columns(df_weather_station)


station_id,lat,lon
70275546404,61.437,-142.904
72224503882,30.212,-85.683
72226613850,32.35,-86.983
70104526649,68.031,-162.903
72074924255,48.35,-122.667


Column,Type,% Null
station_id,string,0.0
lat,double,0.0
lon,double,0.0


Total rows: 2237


In [0]:
# Prepare airport coordinates in radians
airport_radians = (
    df_airports
    .dropna(subset=["lat", "lon"])
    .withColumn("lat_rad", sf.radians("lat"))
    .withColumn("lon_rad", sf.radians("lon"))
    .select("iata_code", "lat", "lon", "lat_rad", "lon_rad")  
)

# Display results
show_df(airport_radians, 5)
show_columns(airport_radians)

iata_code,lat,lon,lat_rad,lon_rad
AMF,-4.166667,141.66667,-0.07272205798419458,2.4725498318362753
HAE,36.230556,-112.66944,0.6323424920281867,-1.9664526943782
LFH,26.64507115,99.54446199297831,0.4650442209956519,1.737378613903828
JNN,60.134167,-45.233612,1.049539207094121,-0.7894754619696173
BNF,57.0,-135.0,0.9948376736367679,-2.356194490192345


Column,Type,% Null
iata_code,string,0.0
lat,double,0.0
lon,double,0.0
lat_rad,double,0.0
lon_rad,double,0.0


Total rows: 9096


In [0]:
# Prepare stations coordinates in radians
stations_radians = (
    df_weather_station
    .dropna(subset=["lat", "lon"])
    .withColumn("st_lat_rad", sf.radians("lat"))
    .withColumn("st_lon_rad", sf.radians("lon"))
    .select(sf.col("station_id").alias("station_id"), "lat", "lon", "st_lat_rad", "st_lon_rad")
)
# Display results
show_df(stations_radians, 5)
show_columns(stations_radians)

station_id,lat,lon,st_lat_rad,st_lon_rad
70275546404,61.437,-142.904,1.0722779325477563,-2.4941453142699768
72224503882,30.212,-85.683,0.5272988736125268,-1.4954504629863015
72226613850,32.35,-86.983,0.5646140130201657,-1.5181397432622277
70104526649,68.031,-162.903,1.1873649434242624,-2.8431937113763226
72074924255,48.35,-122.667,0.8438666933392583,-2.140943033543884


Column,Type,% Null
station_id,string,0.0
lat,double,0.0
lon,double,0.0
st_lat_rad,double,0.0
st_lon_rad,double,0.0


Total rows: 2237


In [0]:
# Cross join (broadcast the small side), compute distance, rank, keep top-3 per airport
airports_stations_cross = (
    sf.broadcast(airport_radians).crossJoin(stations_radians)
    .withColumn("dist_km", haversine_km_expr(sf.col("lat_rad"), sf.col("lon_rad"),
                                             sf.col("st_lat_rad"), sf.col("st_lon_rad")))
    .select(
        sf.col("iata_code"),
        sf.col("station_id").alias("STATION"),
        sf.col("dist_km")
    )
)
station_rank = W.partitionBy("iata_code").orderBy(sf.col("dist_km").asc())
airport_weather_station = (
    airports_stations_cross
    .withColumn("rank", sf.row_number().over(station_rank))
    .filter(sf.col("rank") <= 3)
    .select("iata_code", "STATION", "dist_km", "rank")
)
# Display results
show_df(stations_radians, 5)
show_columns(stations_radians)

station_id,lat,lon,st_lat_rad,st_lon_rad
70275546404,61.437,-142.904,1.0722779325477563,-2.4941453142699768
72224503882,30.212,-85.683,0.5272988736125268,-1.4954504629863015
72226613850,32.35,-86.983,0.5646140130201657,-1.5181397432622277
70104526649,68.031,-162.903,1.1873649434242624,-2.8431937113763226
72074924255,48.35,-122.667,0.8438666933392583,-2.140943033543884


Column,Type,% Null
station_id,string,0.0
lat,double,0.0
lon,double,0.0
st_lat_rad,double,0.0
st_lon_rad,double,0.0


Total rows: 2237


In [0]:
# Prediction timestamp (origin): build local time from schedule (no leakage), then T–2h, then local→UTC
fl = df_flights
crs = (
    fl
    .withColumn("CRS_DEP_TIME_str", sf.lpad(sf.col("CRS_DEP_TIME").cast("string"), 4, "0"))
    .withColumn("dep_hh", sf.col("CRS_DEP_TIME_str").substr(1, 2).cast("int"))
    .withColumn("dep_mm", sf.col("CRS_DEP_TIME_str").substr(3, 2).cast("int"))
    .withColumn("FL_DATE_str", sf.col("FL_DATE").cast("string"))
    .withColumn(
        "dep_local_ts",
        sf.to_timestamp(
            sf.concat_ws(" ", "FL_DATE_str", sf.format_string("%02d:%02d:00", sf.col("dep_hh"), sf.col("dep_mm"))),
            "yyyy-MM-dd HH:mm:ss"
        )
    )
)

# Display results
show_df(crs, 5)
show_columns(crs)


QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME,DIV_AIRPORT_LANDINGS,DIV_REACHED_DEST,DIV_ACTUAL_ELAPSED_TIME,DIV_ARR_DELAY,DIV_DISTANCE,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV1_AIRPORT_SEQ_ID,DIV1_WHEELS_ON,DIV1_TOTAL_GTIME,DIV1_LONGEST_GTIME,DIV1_WHEELS_OFF,DIV1_TAIL_NUM,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV2_AIRPORT_SEQ_ID,DIV2_WHEELS_ON,DIV2_TOTAL_GTIME,DIV2_LONGEST_GTIME,DIV2_WHEELS_OFF,DIV2_TAIL_NUM,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV3_AIRPORT_SEQ_ID,DIV3_WHEELS_ON,DIV3_TOTAL_GTIME,DIV3_LONGEST_GTIME,DIV3_WHEELS_OFF,DIV3_TAIL_NUM,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV4_AIRPORT_SEQ_ID,DIV4_WHEELS_ON,DIV4_TOTAL_GTIME,DIV4_LONGEST_GTIME,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM,YEAR,CRS_DEP_TIME_str,dep_hh,dep_mm,FL_DATE_str,dep_local_ts
2,6,26,7,2022-06-26,OH,20397,OH,N570NN,5404,11481,1148102,31481,ECP,"Panama City, FL",FL,12,Florida,33,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1100,1053,-7.0,0.0,0.0,-1,1100-1159,16.0,1109,1320,9.0,1339,1329,-10.0,0.0,0.0,-1,1300-1359,0.0,null,0.0,99.0,96.0,71.0,1.0,437.0,2,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,1100,11,0,2022-06-26,2022-06-26T11:00:00Z
2,6,30,4,2022-06-30,OH,20397,OH,N556NN,5409,10785,1078502,30785,BTV,"Burlington, VT",VT,50,Vermont,16,11278,1127805,30852,DCA,"Washington, DC",VA,51,Virginia,38,1240,null,null,null,null,null,1200-1259,null,null,null,null,1424,null,null,null,null,null,1400-1459,1.0,A,0.0,104.0,null,null,1.0,437.0,2,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,1240,12,40,2022-06-30,2022-06-30T12:40:00Z
2,6,3,5,2022-06-03,OH,20397,OH,N543EA,5415,14576,1457606,34576,ROC,"Rochester, NY",NY,36,New York,22,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,500,452,-8.0,0.0,0.0,-1,0001-0559,14.0,506,639,5.0,652,644,-8.0,0.0,0.0,-1,0600-0659,0.0,null,0.0,112.0,112.0,93.0,1.0,573.0,3,null,null,null,null,null,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2022,0500,5,0,2022-06-03,2022-06-03T05:00:00Z
2,6,21,2,2022-06-21,OH,20397,OH,N521AE,5434,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,15249,1524906,35249,TLH,"Tallahassee, FL",FL,12,Florida,33,1800,1832,32.0,32.0,1.0,2,1800-1859,52.0,1924,2019,3.0,1932,2022,50.0,50.0,1.0,3,1900-1959,0.0,null,0.0,92.0,110.0,55.0,1.0,386.0,2,32.0,0.0,18.0,0.0,0.0,null,null,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,nu

Column,Type,% Null
QUARTER,int,0.0
MONTH,int,0.0
DAY_OF_MONTH,int,0.0
DAY_OF_WEEK,int,0.0
FL_DATE,string,0.0
OP_UNIQUE_CARRIER,string,0.0
OP_CARRIER_AIRLINE_ID,int,0.0
OP_CARRIER,string,0.0
TAIL_NUM,string,0.7675151841892216
OP_CARRIER_FL_NUM,int,3.1908271632308482E-6


Total rows: 31339836


In [0]:
fl_origin = (
    crs.alias("f")
    .join(df_airports.alias("a"), sf.col("f.ORIGIN")==sf.col("a.iata_code"), "left")
    .withColumn("airport_timezone", sf.col("a.airport_timezone"))                     # expose TZ as a plain column
    .withColumn("prediction_local_ts", sf.expr("dep_local_ts - INTERVAL 2 HOURS"))    # (equiv to T–2h)
    .withColumn(
        "prediction_utc",
        sf.expr("to_utc_timestamp(prediction_local_ts, airport_timezone)")            # use expr() so TZ column works
    )
    .withColumn(
        "flight_id",
        sf.concat_ws("|",
            sf.col("FL_DATE").cast("string"),
            sf.col("OP_UNIQUE_CARRIER"),
            sf.col("OP_CARRIER_FL_NUM"),
            sf.col("ORIGIN"),
            sf.col("DEST")
        )
    )
    .select("flight_id","FL_DATE","ORIGIN","DEST","prediction_local_ts","prediction_utc")
)

# Display results
show_df(fl_origin, 5)
show_columns(fl_origin)

flight_id,FL_DATE,ORIGIN,DEST,prediction_local_ts,prediction_utc
2020-07-16|UA|342|EWR|FLL,2020-07-16,EWR,FLL,2020-07-16T16:50:00Z,2020-07-16T20:50:00Z
2020-07-15|UA|2232|IAD|FLL,2020-07-15,IAD,FLL,2020-07-15T15:45:00Z,2020-07-15T19:45:00Z
2020-07-15|UA|1087|SFO|IAH,2020-07-15,SFO,IAH,2020-07-15T08:55:00Z,2020-07-15T15:55:00Z
2020-07-15|UA|753|SNA|SFO,2020-07-15,SNA,SFO,2020-07-15T06:15:00Z,2020-07-15T13:15:00Z
2020-07-15|UA|306|DEN|ORD,2020-07-15,DEN,ORD,2020-07-15T17:45:00Z,2020-07-15T23:45:00Z


Column,Type,% Null
flight_id,string,0.0
FL_DATE,string,0.0
ORIGIN,string,0.0
DEST,string,0.0
prediction_local_ts,timestamp,3.1908271632308482E-6
prediction_utc,timestamp,3.1908271632308482E-6


Total rows: 31339836


In [0]:
# As-of weather (origin): join candidate stations; filter obs_utc ≤ prediction_utc and within 6h; choose latest by rank/time
origin_candidates = (
    fl_origin.alias("f")
    .join(airport_weather_station.alias("b"), sf.col("f.ORIGIN") == sf.col("b.iata_code"), how="left")
    .select("f.*", sf.col("b.STATION").alias("cand_station"), sf.col("b.rank").alias("station_rank"))
)

# Display results
show_df(origin_candidates)
show_columns(origin_candidates)



flight_id,FL_DATE,ORIGIN,DEST,prediction_local_ts,prediction_utc,cand_station,station_rank
2021-07-17|OH|5502|ABE|CLT,2021-07-17,ABE,CLT,2021-07-17T15:04:00Z,2021-07-17T19:04:00Z,72517014737,1
2021-07-17|OH|5502|ABE|CLT,2021-07-17,ABE,CLT,2021-07-17T15:04:00Z,2021-07-17T19:04:00Z,72032464753,2
2021-07-17|OH|5502|ABE|CLT,2021-07-17,ABE,CLT,2021-07-17T15:04:00Z,2021-07-17T19:04:00Z,72511354786,3
2022-03-20|WN|1657|ALB|BWI,2022-03-20,ALB,BWI,2022-03-20T15:50:00Z,2022-03-20T19:50:00Z,72518014735,1
2022-03-20|WN|1657|ALB|BWI,2022-03-20,ALB,BWI,2022-03-20T15:50:00Z,2022-03-20T19:50:00Z,74499404741,2


Column,Type,% Null
flight_id,string,0.0
FL_DATE,string,0.0
ORIGIN,string,0.0
DEST,string,0.0
prediction_local_ts,timestamp,3.1908271632308482E-6
prediction_utc,timestamp,3.1908271632308482E-6
cand_station,string,0.0
station_rank,int,0.0


Total rows: 94019508


In [0]:
# Restrict to weather rows 'as-of' prediction_utc (no future) and within a bounded lookback window 
weather_required = [
    "HourlyDryBulbTemperature","HourlyDewPointTemperature","HourlyWetBulbTemperature",
    "HourlyPrecipitation","HourlyWindSpeed","HourlyWindDirection","HourlyWindGustSpeed",
    "HourlyVisibility","HourlyRelativeHumidity","HourlyStationPressure","HourlySeaLevelPressure",
    "HourlyAltimeterSetting","HourlySkyConditions","HourlyPresentWeatherType"
]
wx_present = [c for c in weather_required if c in weather_best.columns]
weather = weather_best.select(
    "STATION",
    sf.col("obs_utc"),
    *[sf.col(c) for c in wx_present]
)

# weather_cols = ["STATION", sf.col("DATE").cast("timestamp").alias("obs_utc")] + [sf.col(c) for c in wx_present]
# weather = df_weather.select(*weather_cols)

# In origin_candidates, also carry the distance so we can publish origin_station_dis
origin_candidates = (
    fl_origin.alias("f")
    .join(airport_weather_station.alias("b"), sf.col("f.ORIGIN") == sf.col("b.iata_code"), how="left")
    .select("f.*",
            sf.col("b.STATION").alias("cand_station"),
            sf.col("b.rank").alias("station_rank"),
            sf.col("b.dist_km").alias("cand_station_dis_km"))
)

# 6-hour lookback 
weather_join = (
    origin_candidates.alias("x")
    .join(
        weather.alias("w"),
        on=[
            sf.col("w.STATION") == sf.col("x.cand_station"),
            sf.col("w.obs_utc") <= sf.col("x.prediction_utc"),
            sf.col("w.obs_utc") >= sf.expr("timestampadd(HOUR, -6, x.prediction_utc)")
        ],
        how="left"
    )
)

# Display results
show_df(weather_join, 5)
show_columns(weather_join)

flight_id,FL_DATE,ORIGIN,DEST,prediction_local_ts,prediction_utc,cand_station,station_rank,cand_station_dis_km,STATION,obs_utc,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyWetBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyWindDirection,HourlyWindGustSpeed,HourlyVisibility,HourlyRelativeHumidity,HourlyStationPressure,HourlySeaLevelPressure,HourlyAltimeterSetting,HourlySkyConditions,HourlyPresentWeatherType
2021-12-20|OH|5353|PNS|CLT,2021-12-20,PNS,CLT,2021-12-20T03:40:00Z,2021-12-20T09:40:00Z,A0002453848,3,22.18721781147289,A0002453848,2021-12-20T06:53:00Z,10.0,3.9,7.2,0.0,2.6,30.0,null,16.093,66.0,1017.2,1021.6,1021.3,CLR:00,null
2021-12-20|OH|5353|PNS|CLT,2021-12-20,PNS,CLT,2021-12-20T03:40:00Z,2021-12-20T09:40:00Z,A0002453848,3,22.18721781147289,A0002453848,2021-12-20T04:53:00Z,10.0,3.3,6.9,0.0,2.6,10.0,null,16.093,63.0,1016.9,1021.4,1021.0,CLR:00,null
2021-12-20|OH|5353|PNS|CLT,2021-12-20,PNS,CLT,2021-12-20T03:40:00Z,2021-12-20T09:40:00Z,A0002453848,3,22.18721781147289,A0002453848,2021-12-20T05:53:00Z,10.0,3.9,7.2,0.0,3.6,350.0,null,16.093,66.0,1017.2,1021.6,1021.3,CLR:00,null
2021-12-20|OH|5353|PNS|CLT,2021-12-20,PNS,CLT,2021-12-20T03:40:00Z,2021-12-20T09:40:00Z,A0002453848,3,22.18721781147289,A0002453848,2021-12-20T03:53:00Z,10.0,3.3,6.9,0.0,3.1,20.0,null,16.093,63.0,1016.6,1021.1,1020.7,CLR:00,null
2021-12-20|OH|5353|PNS|CLT,2021-12-20,PNS,CLT,2021-12-20T03:40:00Z,2021-12-20T09:40:00Z,A0002453848,3,22.18721781147289,A0002453848,2021-12-20T07:53:00Z,10.0,3.9,7.2,0.0,3.1,20.0,null,16.093,66.0,1017.2,1021.7,1021.3,CLR:00,null


Column,Type,% Null
flight_id,string,0.0
FL_DATE,string,0.0
ORIGIN,string,0.0
DEST,string,0.0
prediction_local_ts,timestamp,3.785509250275451E-7
prediction_utc,timestamp,3.785509250275451E-7
cand_station,string,0.0
station_rank,int,0.0
cand_station_dis_km,double,0.0
STATION,string,1.0768679804860328


Total rows: 792495752


In [0]:
# Station selection window, prefer lower station_rank (1, then 2, then 3), and the latest obs_utc within the window
window = W.partitionBy("flight_id").orderBy(sf.col("station_rank").asc(), sf.col("obs_utc").desc())
origin_asof = (
    weather_join
    .withColumn("rn", sf.row_number().over(window))
    .filter(sf.col("rn") == 1)
    .withColumn("asof_minutes", sf.floor((sf.unix_timestamp("prediction_utc") - sf.unix_timestamp("obs_utc"))/60.0))
    .select(
        "flight_id","ORIGIN","prediction_utc",
        sf.col("cand_station").alias("origin_station_id"),
        sf.col("cand_station_dis_km").alias("origin_station_dis"),
        sf.col("obs_utc").alias("origin_obs_utc"),
        "asof_minutes",
        *wx_present,
        "station_rank"
    )
)
# Display results
# show_df(origin_asof, 5)
# show_columns(origin_asof)

In [0]:
# Origin station lat/lon
origin_asof_enriched = (
    origin_asof.alias("o")
    .join(
        df_weather_station.select(
            sf.col("station_id").alias("STATION"),
            sf.col("lat").alias("origin_station_lat"),
            sf.col("lon").alias("origin_station_lon")
        ).alias("s"),
        sf.col("o.origin_station_id")==sf.col("s.STATION"),
        "left"
    )
)

need_from_w = ["flight_id","prediction_utc","origin_obs_utc","asof_minutes",
               "origin_station_id","origin_station_dis","origin_station_lat","origin_station_lon"] + wx_present
origin_asof_enriched = origin_asof_enriched.select(*[c for c in need_from_w if c in origin_asof_enriched.columns])


# Airport lat/lon (origin & dest)
air_min = df_airports.select(
    "iata_code",
    sf.col("lat").alias("airport_lat"),
    sf.col("lon").alias("airport_lon"),
    sf.col("airport_type")
)
origin_air_geo = air_min.select(
    sf.col("iata_code").alias("ORIGIN"),
    sf.col("airport_lat").alias("origin_airport_lat"),
    sf.col("airport_lon").alias("origin_airport_lon"),
    sf.col("airport_type").alias("origin_type")   
)
dest_air_geo = air_min.select(
    sf.col("iata_code").alias("DEST"),
    sf.col("airport_lat").alias("dest_airport_lat"),
    sf.col("airport_lon").alias("dest_airport_lon"),
    sf.col("airport_type").alias("dest_type")     # <-- new
)

# Dest station (rank-1) for location helpers (no dest weather to avoid leakage)
dest_rank1 = (
    airport_weather_station
    .filter(sf.col("rank")==1)
    .select(
        sf.col("iata_code").alias("DEST"),
        sf.col("STATION").alias("dest_station_id"),
        sf.col("dist_km").alias("dest_station_dis")
    )
)
dest_station_geo = (
    dest_rank1.alias("d")
    .join(
        df_weather_station.select(
            sf.col("station_id").alias("STATION"),
            sf.col("lat").alias("dest_station_lat"),
            sf.col("lon").alias("dest_station_lon")
        ).alias("s"),
        sf.col("d.dest_station_id")==sf.col("s.STATION"),
        "left"
    )
    .select("DEST","dest_station_id","dest_station_dis","dest_station_lat","dest_station_lon")
)

In [0]:
# Rebuild a stable flight_id on the raw flights for the final join
flights_keyed = (
    df_flights
    .withColumn(
        "flight_id",
        sf.concat_ws("|",
            sf.col("FL_DATE").cast("string"),
            sf.col("OP_UNIQUE_CARRIER"),
            sf.col("OP_CARRIER_FL_NUM"),
            sf.col("ORIGIN"),
            sf.col("DEST")
        )
    )
)

# Display results
# show_df(flights_keyed, 5)
# show_columns(flights_keyed)

In [0]:
# Assemble final dataset: one row per flight
final_joined = (
    flights_keyed.alias("f")
    .join(origin_asof_enriched.alias("w"), "flight_id", "left")
    .join(origin_air_geo, ["ORIGIN"], "left")
    .join(dest_air_geo,   ["DEST"],   "left")
    .join(dest_station_geo, ["DEST"], "left")
)

# Column groups
model_inputs = [
    # Flight & schedule
    "FL_DATE","YEAR","QUARTER","MONTH","DAY_OF_MONTH","DAY_OF_WEEK",
    "OP_UNIQUE_CARRIER","OP_CARRIER","OP_CARRIER_FL_NUM","TAIL_NUM",
    "CRS_DEP_TIME","CRS_ARR_TIME","CRS_ELAPSED_TIME",
    "ORIGIN","ORIGIN_AIRPORT_ID","ORIGIN_CITY_NAME","ORIGIN_STATE_ABR",
    "DEST","DEST_AIRPORT_ID","DEST_CITY_NAME","DEST_STATE_ABR",
    "DISTANCE","DISTANCE_GROUP",
    # Origin weather (as-of T–2h)
] + wx_present + [
    # Location helpers
    "origin_station_lat","origin_station_lon","origin_airport_lat","origin_airport_lon",
    "dest_station_lat","dest_station_lon","dest_airport_lat","dest_airport_lon",
    "origin_station_dis","dest_station_dis", "origin_type","dest_type"
]

labels_eval = ["DEP_DEL15","DEP_DELAY","ARR_DEL15","ARR_DELAY"]
post_flight = [
    "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY",
    "DEP_TIME","ARR_TIME","TAXI_OUT","TAXI_IN","WHEELS_OFF","WHEELS_ON","ACTUAL_ELAPSED_TIME","AIR_TIME"
]
flags = ["CANCELLED","CANCELLATION_CODE","DIVERTED"]

provenance = ["flight_id","prediction_utc","origin_obs_utc","asof_minutes","origin_station_id","dest_station_id"]

# Keep only columns that exist (different weather slices may miss some)
def present(cols, df_cols): 
    s = set(df_cols); 
    return [c for c in cols if c in s]

keep = provenance \
     + present(model_inputs, final_joined.columns) \
     + present(labels_eval, final_joined.columns) \
     + present(post_flight, final_joined.columns) \
     + present(flags, final_joined.columns)

final_curated = final_joined.select(*keep)


# Optional: limit to an end date if configured
max_year = cfg.get("max_year")
max_month = cfg.get("max_month")

if max_year is not None:
    if max_month is None:
        max_month = 12
    final_curated = final_curated.filter(
        (sf.col("YEAR") < max_year)
        | ((sf.col("YEAR") == max_year) & (sf.col("MONTH") <= max_month))
    )

# Persist for the team using the configured output path
out_path = cfg["joined_output"]
(final_curated
 .write
 .format("parquet")
 .mode("overwrite")
 .save(out_path))

print(f"Wrote joined dataset to: {out_path}")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3740109152944026>, line 68
     62 # Persist for the team using the configured output path
     63 out_path = cfg["joined_output"]
     64 (final_curated
     65  .write
     66  .format("parquet")
     67  .mode("overwrite")
---> 68  .save(out_path))
     70 print(f"Wrote joined dataset to: {out_path}")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1734, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1732     self._

In [0]:
# 1. Repartition the DataFrame to reduce shuffle pressure
final_curated = final_curated.repartition(200)

# 2. Write with overwrite mode as before
final_curated.write.format("parquet").mode("overwrite").save(out_path)

### 1Y Join from 5Y Data

In [0]:
# Build a 1Y (2015) subset only when running the 5Y config
if DATA_SLICE == "5Y":
    df_joined_5Y = spark.read.parquet(DATA_CONFIG["5Y"]["joined_output"])
    print("Source row count (5Y):", df_joined_5Y.count())

    df_2015 = df_joined_5Y.filter(sf.col("YEAR") == 2015)
    print("Destination row count (1Y 2015):", df_2015.count())

    df_2015.write.mode("overwrite").parquet("dbfs:/student-groups/Group_4_4/JOINED_1Y_2015.parquet")

## Get stats

In [0]:
# RUN STATS: summary of this join job

import pandas as pd

job_end = datetime.now()
runtime_seconds = (job_end - job_start).total_seconds()
runtime_minutes = round(runtime_seconds / 60, 2)

print(f"Job end:   {job_end.isoformat()}")
print(f"Runtime:   {runtime_seconds:.1f} seconds ({runtime_minutes} minutes)")

# --- Dataset-level stats (final_curated) ---

# Number of samples and features
row_count = final_curated.count()              # triggers a Spark job
feature_count = len(final_curated.columns)

# Date coverage (uses FL_DATE or change to YEAR/MONTH if you prefer)
date_bounds = final_curated.agg(
    sf.min("FL_DATE").alias("min_date"),
    sf.max("FL_DATE").alias("max_date")
).collect()[0]

data_start_date = date_bounds["min_date"]
data_end_date = date_bounds["max_date"]

# --- File sizes (inputs/outputs) ---

INPUT_PATHS = {
    "flights": cfg["flights_path"],
    "weather": cfg["weather_path"],
    # You can add more here if you want:
    # "airport_codes": "dbfs:/path/to/airport-codes.csv",
    # "airports_with_tz": "dbfs:/path/to/airports.csv",
    # "stations": "dbfs:/path/to/stations_with_neighbors.parquet",
}

OUTPUT_PATHS = {
    "joined_output": out_path,   # out_path was used when writing final_curated
}

file_rows = []

for name, path in INPUT_PATHS.items():
    size_bytes = get_dir_size(path)
    file_rows.append({
        "role": "input",
        "name": name,
        "path": path,
        "size_mb": round(size_bytes / (1024 * 1024), 2),
    })

for name, path in OUTPUT_PATHS.items():
    size_bytes = get_dir_size(path)
    file_rows.append({
        "role": "output",
        "name": name,
        "path": path,
        "size_mb": round(size_bytes / (1024 * 1024), 2),
    })

files_df = pd.DataFrame(file_rows)

# --- High-level run summary ---

summary_rows = [
    {"metric": "data_slice",        "value": DATA_SLICE},
    {"metric": "job_start",         "value": job_start.isoformat()},
    {"metric": "job_end",           "value": job_end.isoformat()},
    {"metric": "runtime_minutes",   "value": runtime_minutes},
    {"metric": "row_count",         "value": row_count},
    {"metric": "feature_count",     "value": feature_count},
    {"metric": "data_start_date",   "value": str(data_start_date)},
    {"metric": "data_end_date",     "value": str(data_end_date)},
]

summary_df = pd.DataFrame(summary_rows)

display(summary_df)
display(files_df)


Job end:   2025-12-15T07:26:37.315842
Runtime:   47767.7 seconds (796.13 minutes)


/databricks/spark/python/pyspark/sql/pandas/conversion.py:516: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected bytes, got a 'float' object
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


metric,value
data_slice,FUTURE
job_start,2025-12-14T18:10:29.575959
job_end,2025-12-15T07:26:37.315842
runtime_minutes,796.13
row_count,31339836
feature_count,75
data_start_date,2020-01-01
data_end_date,2024-12-31


role,name,path,size_mb
input,flights,dbfs:/student-groups/Group_4_4/future_joins/BTS_OnTime/parquet_airlines_data_2020_2024_std.parquet,975.11
input,weather,dbfs:/student-groups/Group_4_4/future_joins/NOAA/LCDv2_weather_data_2020_2024_std.parquet,21917.69
output,joined_output,dbfs:/student-groups/Group_4_4/JOINED_FUTURE.parquet,2489.98


## Preview

In [0]:
df_check = spark.read.parquet(out_path)
display(df_check.limit(10))

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can